In [ ]:
from get_data import scrape_companies_info
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import pandas as pd

In [ ]:
# company_info = StructType([
#     StructField("cik", IntegerType(), nullable=False),
#     StructField("cik_str", StringType(), nullable=False),
#     StructField("ticker", StringType(), nullable=False),
#     StructField("name", StringType()),
#     StructField("sector", StringType()),
#     StructField("industry", StringType())
# ])

In [ ]:
ciks = scrape_companies_info.get_ciks()

In [ ]:
# # Create a SparkSession
# spark = SparkSession.builder.appName("DictToDataFrame").getOrCreate()

# # Sample dictionary data
# data = {'0': {'cik': 789019, 'cik_str': '0000789019', 'ticker': 'MSFT', 'name': 'MICROSOFT CORP'},
#         '1': {'cik': 320193, 'cik_str': '0000320193', 'ticker': 'AAPL', 'name': 'Apple Inc.'}}

# # Function to convert a dictionary value to a row
# def create_row(value):
#   return (value['cik'], value['cik_str'], value['ticker'], value['name'])

# # Create a list of rows
# rows = [create_row(value) for _, value in data.items()]

# # Create a DataFrame from the list of rows
# df = spark.createDataFrame(rows, ["cik", "cik_str", "ticker", "name"])

# df.show(n=10)

In [9]:
def company_info_update():
    path = "../data/finance_data/company_info.csv"

    df = pd.DataFrame(ciks).T
    df = df.set_index("cik")
    df.to_csv(path)

In [30]:
df.filter({"ticker":"AAPL"})

,ticker
cik,
789019,MSFT
320193,AAPL
1045810,NVDA
1652044,GOOGL
1018724,AMZN
...,...
1838163,BNAIW
1903392,CLRCR
1903392,CLRCU


In [ ]:
def company_info_get(df, filter:dict, columns:list=None, get_first:bool=True):
    
    

In [25]:
df.iloc[:].index

Index([ 789019,  320193, 1045810, 1652044, 1018724, 1326801, 1067983, 1046179,
         59478, 1730168,
       ...
       1764046, 1851182, 1851182,  886982, 1532619, 1838163, 1903392, 1903392,
       1903392, 1298946],
      dtype='object', name='cik', length=10288)

In [ ]:
def company_info_get(session, filter:dict, columns:list=None, get_first:bool=True):
    """
    Search for company info by filter and return the result as a dictionary.
    
    :param:
        session: SQLAlchemy session object
        filter: The filter (in dict) to search for
        columns: List of columns to return (default is all columns)
        get_first: If True, return only first object
    :return: Dictionary containing company info (return list if have many result) if found, otherwise None
    """
    query = session.query(CompanyInfo).filter_by(**filter)
    
    if columns:
        orm_columns = [getattr(CompanyInfo, column) for column in columns]
        query = query.options(load_only(*orm_columns))
        
    companies = query.all()
    
    # Close the session
    session.close()
    
    if len(companies)==1:
        return companies[0].__dict__
    elif companies:
        if get_first:
            return companies[0].__dict__
        return [ company.__dict__ for company in companies ]
    else:
        return None